# 🏇 競馬AI データパイプライン（Google Colab版）

このノートブックは以下を実行します：
1. **スクレイピング**: netkeiba.comからレースデータを収集
2. **データ前処理**: 欠損値処理、型変換、外れ値除去
3. **特徴量生成**: 過去走データから予測用特徴量を計算
4. **SQLite保存**: データベースに保存してアプリから利用可能に

## 📋 必要なライブラリ

In [ ]:
# Google Colab環境のセットアップ
print("📦 ライブラリをインストール中...")
try:
    !pip install beautifulsoup4 requests pandas numpy scikit-learn lightgbm tqdm -q
    print("✅ インストール完了")
except Exception as e:
    print(f"❌ インストールエラー: {e}")
    print("💡 ランタイムを再起動して再試行してください")
    raise

try:
    import os
    import sys
    import sqlite3
    import pandas as pd
    import numpy as np
    from datetime import datetime, timedelta
    from tqdm import tqdm
    import pickle
    import json
    print("✅ ライブラリのインポート完了")
except ImportError as e:
    print(f"❌ インポートエラー: {e}")
    print("💡 上記のpipインストールが正しく実行されたか確認してください")
    raise

## 🔧 Google Driveマウント（オプション）

データベースをGoogle Driveに保存する場合

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Driveマウント成功")
except Exception as e:
    print(f"❌ Google Driveマウントエラー: {e}")
    print("💡 ブラウザでGoogleアカウントへのアクセスを許可してください")
    raise

# 作業ディレクトリの設定
WORK_DIR = '/content/drive/MyDrive/keiba-ai'

try:
    os.makedirs(WORK_DIR, exist_ok=True)
    os.chdir(WORK_DIR)
    print(f"✅ 作業ディレクトリ: {WORK_DIR}")
except Exception as e:
    print(f"❌ ディレクトリ作成エラー: {e}")
    print(f"💡 Google Driveの容量を確認してください")
    # フォールバック: ローカル作業ディレクトリを使用
    WORK_DIR = '/content'
    os.chdir(WORK_DIR)
    print(f"⚠️ フォールバック: {WORK_DIR} を使用")

## 📥 GitHubからコードをクローン

In [ ]:
# GitHubリポジトリのクローン（初回のみ）
REPO_URL = "https://github.com/YOUR_USERNAME/dai-keiba.git"  # ⚠️ 実際のURLに変更してください

print("📥 GitHubリポジトリをクローン中...")

try:
    if not os.path.exists('dai-keiba'):
        !git clone {REPO_URL}
        if not os.path.exists('dai-keiba'):
            raise Exception("リポジトリのクローンに失敗しました")
        print("✅ リポジトリをクローンしました")
    else:
        print("✅ リポジトリは既に存在します")
        # 最新版を取得
        try:
            !cd dai-keiba && git pull
            print("✅ 最新版を取得しました")
        except Exception as e:
            print(f"⚠️ git pullに失敗: {e}")
            print("💡 既存のリポジトリを使用します")
except Exception as e:
    print(f"❌ Gitクローンエラー: {e}")
    print("💡 REPO_URLを正しいURLに変更してください")
    print("💡 または、ファイルを手動でアップロードしてください")
    raise

# パスの追加
try:
    sys.path.append('dai-keiba')
    sys.path.append('dai-keiba/scraper')
    sys.path.append('dai-keiba/ml')
    print("✅ Pythonパスを追加しました")
except Exception as e:
    print(f"❌ パス追加エラー: {e}")
    raise

## 🕷️ ステップ1: スクレイピング

### JRAデータの収集

In [ ]:
try:
    from scraper import auto_scraper
    print("✅ スクレイピングモジュールのインポート成功")
except ImportError as e:
    print(f"❌ インポートエラー: {e}")
    print("💡 dai-keiba/scraper/ ディレクトリが存在するか確認してください")
    raise

# 設定
START_DATE = "2023-01-01"  # 開始日
END_DATE = datetime.now().strftime("%Y-%m-%d")  # 今日まで
MODE = "JRA"  # JRA または NAR

print(f"📅 収集期間: {START_DATE} ～ {END_DATE}")
print(f"🏇 モード: {MODE}")

csv_path = f"database_{MODE.lower()}.csv"
db_path = "keiba_data.db"  # SQLiteデータベースパス

try:
    # ====================================
    # 既存データの確認と欠落分析
    # ====================================
    print("\n🔍 既存データを確認中...")

    # 既存race_idを取得（SQLiteまたはCSVから）
    existing_race_ids = auto_scraper.get_existing_race_ids(
        mode=MODE,
        db_path=db_path if os.path.exists(db_path) else None,
        csv_path=csv_path if os.path.exists(csv_path) else None
    )

    if existing_race_ids:
        print(f"✅ 既存データ: {len(existing_race_ids)}レース")

        # 欠落レースの分析
        from datetime import datetime
        start_dt = datetime.strptime(START_DATE, "%Y-%m-%d")
        end_dt = datetime.strptime(END_DATE, "%Y-%m-%d")

        missing_info = auto_scraper.find_missing_races(
            start_dt,
            end_dt,
            existing_race_ids,
            mode=MODE
        )

        print(f"\n📊 データカバレッジ分析:")
        print(f"   対象期間: {missing_info['total_days']}日間")
        print(f"   週末日数: {missing_info['weekend_days']}日")
        print(f"   既存レース日: {missing_info['existing_race_dates']}日")
        print(f"   カバレッジ率: {missing_info['coverage_rate']:.1%}")

        if missing_info['missing_weekend_dates']:
            print(f"\n⚠️ 欠落している可能性のある週末: {len(missing_info['missing_weekend_dates'])}日")
            print("   最初の10日:")
            for d in missing_info['missing_weekend_dates'][:10]:
                print(f"     - {d.strftime('%Y-%m-%d (%a)')}")
            if len(missing_info['missing_weekend_dates']) > 10:
                print(f"     ... 他 {len(missing_info['missing_weekend_dates']) - 10}日")

            # ユーザーに確認
            print(f"\n💡 これらの日付を含む期間でスクレイピングを実行します")
            print(f"💡 既存のrace_idはスキップされます")
        else:
            print(f"\n✅ 欠落データなし！すべての週末がカバーされています")
    else:
        print("ℹ️ 既存データが見つかりません（初回スクレイピング）")
        print(f"💡 全期間 {START_DATE} ～ {END_DATE} をスクレイピングします")

    # ====================================
    # スクレイピング実行
    # ====================================
    print(f"\n⏳ スクレイピング開始...（数時間かかる場合があります）")
    print("💡 既存のrace_idは自動的にスキップされます")
    print("💡 ネットワークエラーが発生した場合は自動的にリトライします\n")

    # auto_scraper.main()を呼び出し（内部でexisting_race_idsを使用）
    auto_scraper.main(
        start_date_arg=START_DATE,
        end_date_arg=END_DATE,
        mode_arg=MODE,
        source_arg="jra" if MODE == "JRA" else None
    )

    # スクレイピング結果の確認
    if os.path.exists(csv_path):
        print(f"\n✅ スクレイピング完了: {csv_path}")
        df = pd.read_csv(csv_path)
        print(f"   総レコード数: {len(df):,}")
        print(f"   ユニークrace_id: {df['race_id'].nunique():,}")

        # データ品質チェック
        if len(df) < 100:
            print("⚠️ データ量が少ないです（100件未満）")
            print("💡 より長い期間のデータを収集することを推奨します")

        # 必須カラムのチェック
        required_cols = ['race_id', 'horse_id', 'date']
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"必須カラムが不足: {missing_cols}")

        # 新規追加されたrace_idの数
        new_race_ids = set(df['race_id'].astype(str)) - existing_race_ids
        if new_race_ids:
            print(f"\n🆕 新規追加されたレース: {len(new_race_ids)}件")
        else:
            print(f"\n💡 新規レースは追加されませんでした（すべて既存）")

        print("✅ データ品質チェック完了")
    else:
        raise FileNotFoundError(
            f"{csv_path} が見つかりません。\n"
            "💡 スクレイピングが正常に完了していない可能性があります"
        )

except FileNotFoundError as e:
    print(f"❌ ファイルエラー: {e}")
    raise
except ValueError as e:
    print(f"❌ データ検証エラー: {e}")
    raise
except Exception as e:
    print(f"❌ スクレイピングエラー: {e}")
    print("💡 ネットワーク接続を確認してください")
    print("💡 または、手動でCSVファイルをアップロードしてください")
    import traceback
    print("\n詳細なエラー情報:")
    traceback.print_exc()
    raise

## 🔄 ステップ2: 特徴量生成

In [ ]:
try:
    from ml.feature_engineering import calculate_features
    print("✅ 特徴量生成モジュールのインポート成功")
except ImportError as e:
    print(f"❌ インポートエラー: {e}")
    print("💡 dai-keiba/ml/feature_engineering.py が存在するか確認してください")
    raise

# 設定
USE_VENUE_FEATURES = True  # 会場特性特徴量を使用

input_csv = f"database_{MODE.lower()}.csv"
output_csv = f"processed_data_{MODE.lower()}.csv"

print(f"⚙️ 特徴量生成開始...")
print(f"   入力: {input_csv}")
print(f"   出力: {output_csv}")
print(f"   会場特性: {USE_VENUE_FEATURES}")

try:
    # 入力ファイルの存在確認
    if not os.path.exists(input_csv):
        raise FileNotFoundError(
            f"入力CSVファイルが見つかりません: {input_csv}\n"
            "💡 スクレイピングを先に実行してください"
        )
    
    # データの前チェック
    df_input = pd.read_csv(input_csv)
    print(f"   入力データ: {len(df_input):,}件")
    
    if len(df_input) == 0:
        raise ValueError("入力データが空です")
    
    # 特徴量生成
    print("⏳ 特徴量を計算中...（数分かかる場合があります）")
    calculate_features(
        input_csv,
        output_csv,
        lambda_decay=0.2,
        use_venue_features=USE_VENUE_FEATURES
    )
    
    # 出力データの検証
    if not os.path.exists(output_csv):
        raise FileNotFoundError(f"出力ファイルが生成されませんでした: {output_csv}")
    
    df_output = pd.read_csv(output_csv)
    print(f"\n✅ 特徴量生成完了: {output_csv}")
    print(f"   出力データ: {len(df_output):,}件")
    print(f"   特徴量数: {len(df_output.columns)}個")
    
    # 必須カラムの確認
    required_output_cols = ['race_id', 'horse_id', 'date', 'target_win']
    missing_cols = [col for col in required_output_cols if col not in df_output.columns]
    if missing_cols:
        raise ValueError(f"出力データに必須カラムが不足: {missing_cols}")
    
    # 欠損値チェック
    null_counts = df_output.isnull().sum()
    high_null_cols = null_counts[null_counts > len(df_output) * 0.5]
    if len(high_null_cols) > 0:
        print(f"⚠️ 欠損値が50%以上のカラム: {len(high_null_cols)}個")
        for col, count in high_null_cols.head(5).items():
            print(f"   - {col}: {count}/{len(df_output)} ({count/len(df_output)*100:.1f}%)")
    
    print("✅ データ品質チェック完了")
    
except FileNotFoundError as e:
    print(f"❌ ファイルエラー: {e}")
    raise
except ValueError as e:
    print(f"❌ データ検証エラー: {e}")
    raise
except Exception as e:
    print(f"❌ 特徴量生成エラー: {e}")
    print("💡 入力データの形式を確認してください")
    import traceback
    print("\n詳細なエラー情報:")
    traceback.print_exc()
    raise

## 💾 ステップ3: SQLiteデータベースに保存

In [ ]:
# SQLiteデータベースの作成
DB_PATH = "keiba_data.db"

print(f"💾 データベース作成: {DB_PATH}")

try:
    # データの読み込み
    if not os.path.exists(output_csv):
        raise FileNotFoundError(
            f"特徴量CSVファイルが見つかりません: {output_csv}\n"
            "💡 特徴量生成を先に実行してください"
        )
    
    print(f"📥 データを読み込み中...")
    df_processed = pd.read_csv(output_csv)
    
    print(f"   レコード数: {len(df_processed):,}")
    print(f"   特徴量数: {len(df_processed.columns)}")
    
    if len(df_processed) == 0:
        raise ValueError("処理済みデータが空です")
    
    # SQLite接続
    print(f"🔗 SQLiteに接続中...")
    conn = sqlite3.connect(DB_PATH)
    
    # テーブル名の決定
    table_name = f"processed_data_{MODE.lower()}"
    
    print(f"💾 テーブルに保存中: {table_name}")
    
    # DataFrameをSQLiteに保存
    df_processed.to_sql(
        table_name,
        conn,
        if_exists='replace',  # 既存テーブルを置き換え
        index=False,
        chunksize=1000  # メモリ効率化
    )
    
    # インデックスの作成（高速化）
    print("🔍 インデックスを作成中...")
    cursor = conn.cursor()
    
    # race_idインデックス
    try:
        cursor.execute(f"CREATE INDEX IF NOT EXISTS idx_race_id ON {table_name}(race_id)")
        print("   ✅ race_id インデックス")
    except Exception as e:
        print(f"   ⚠️ race_id インデックス作成失敗: {e}")
    
    # horse_idインデックス
    try:
        cursor.execute(f"CREATE INDEX IF NOT EXISTS idx_horse_id ON {table_name}(horse_id)")
        print("   ✅ horse_id インデックス")
    except Exception as e:
        print(f"   ⚠️ horse_id インデックス作成失敗: {e}")
    
    # dateインデックス
    try:
        cursor.execute(f"CREATE INDEX IF NOT EXISTS idx_date ON {table_name}(date)")
        print("   ✅ date インデックス")
    except Exception as e:
        print(f"   ⚠️ date インデックス作成失敗: {e}")
    
    conn.commit()
    
    print(f"\n✅ データベース保存完了: {table_name}")
    
    # 確認クエリ
    print("🔍 データベースを検証中...")
    result = pd.read_sql_query(f"SELECT COUNT(*) as count FROM {table_name}", conn)
    saved_count = result['count'].iloc[0]
    print(f"   保存されたレコード数: {saved_count:,}")
    
    if saved_count != len(df_processed):
        print(f"⚠️ 警告: 保存数が一致しません（期待: {len(df_processed):,}, 実際: {saved_count:,}）")
    else:
        print("   ✅ レコード数が一致しました")
    
    # データベースサイズの確認
    db_size = os.path.getsize(DB_PATH)
    print(f"   データベースサイズ: {db_size / 1024 / 1024:.2f} MB")
    
    conn.close()
    print("✅ データベース接続をクローズしました")
    
except FileNotFoundError as e:
    print(f"❌ ファイルエラー: {e}")
    raise
except ValueError as e:
    print(f"❌ データ検証エラー: {e}")
    raise
except sqlite3.Error as e:
    print(f"❌ SQLiteエラー: {e}")
    print("💡 データベースファイルの権限を確認してください")
    if 'conn' in locals():
        conn.close()
    raise
except Exception as e:
    print(f"❌ データベース保存エラー: {e}")
    import traceback
    print("\n詳細なエラー情報:")
    traceback.print_exc()
    if 'conn' in locals():
        conn.close()
    raise

## 📊 ステップ4: データ検証

In [ ]:
# データベースからサンプルを読み込んで確認
print("🔍 データベースを検証中...")

try:
    if not os.path.exists(DB_PATH):
        raise FileNotFoundError(
            f"データベースファイルが見つかりません: {DB_PATH}\n"
            "💡 SQLite保存を先に実行してください"
        )
    
    conn = sqlite3.connect(DB_PATH)
    
    # テーブル一覧
    print("\n📋 テーブル一覧:")
    tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
    if len(tables) == 0:
        print("⚠️ テーブルが存在しません")
    else:
        for table_name in tables['name']:
            print(f"   • {table_name}")
    
    # 現在のテーブルが存在するか確認
    table_name = f"processed_data_{MODE.lower()}"
    if table_name not in tables['name'].values:
        raise ValueError(
            f"テーブル '{table_name}' が見つかりません\n"
            "💡 SQLite保存を先に実行してください"
        )
    
    # サンプルデータ
    print(f"\n📊 サンプルデータ（先頭5件）:")
    sample = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 5", conn)
    print(sample)
    
    # 統計情報
    print(f"\n📈 統計情報:")
    stats_query = f"""
        SELECT 
            COUNT(*) as total_records,
            COUNT(DISTINCT race_id) as unique_races,
            COUNT(DISTINCT horse_id) as unique_horses,
            MIN(date) as earliest_date,
            MAX(date) as latest_date,
            AVG(target_win) as win_rate
        FROM {table_name}
    """
    
    try:
        stats = pd.read_sql_query(stats_query, conn)
        print(stats)
        
        # 統計情報の検証
        total_records = stats['total_records'].iloc[0]
        win_rate = stats['win_rate'].iloc[0]
        
        if total_records < 1000:
            print(f"\n⚠️ データ量が少ないです（{total_records:,}件）")
            print("💡 推奨: 5000件以上のデータで学習することを推奨します")
        
        if win_rate < 0.03 or win_rate > 0.20:
            print(f"\n⚠️ 勝率が異常です: {win_rate:.2%}")
            print("💡 通常、勝率は5-10%程度です")
        else:
            print(f"\n✅ 勝率は正常範囲内です: {win_rate:.2%}")
        
    except Exception as e:
        print(f"⚠️ 統計情報の取得に失敗: {e}")
    
    # カラムリスト
    print(f"\n📝 カラム一覧（最初の10個）:")
    pragma_query = f"PRAGMA table_info({table_name})"
    columns_info = pd.read_sql_query(pragma_query, conn)
    for idx, row in columns_info.head(10).iterrows():
        print(f"   {idx+1}. {row['name']} ({row['type']})")
    
    if len(columns_info) > 10:
        print(f"   ... 他 {len(columns_info) - 10}個")
    
    print(f"\n✅ データベース検証完了")
    print(f"   総カラム数: {len(columns_info)}")
    
    conn.close()
    
except FileNotFoundError as e:
    print(f"❌ ファイルエラー: {e}")
    raise
except ValueError as e:
    print(f"❌ 検証エラー: {e}")
    raise
except sqlite3.Error as e:
    print(f"❌ SQLiteエラー: {e}")
    if 'conn' in locals():
        conn.close()
    raise
except Exception as e:
    print(f"❌ データベース検証エラー: {e}")
    import traceback
    print("\n詳細なエラー情報:")
    traceback.print_exc()
    if 'conn' in locals():
        conn.close()
    raise

## 📤 ステップ5: データベースのダウンロード

作成したデータベースをローカルにダウンロードして、アプリケーションで使用します。

In [ ]:
from google.colab import files

# データベースファイルをダウンロード
print("📥 データベースをダウンロード中...")

try:
    if not os.path.exists(DB_PATH):
        raise FileNotFoundError(
            f"データベースファイルが見つかりません: {DB_PATH}\n"
            "💡 SQLite保存を先に実行してください"
        )
    
    # ファイルサイズの確認
    file_size = os.path.getsize(DB_PATH)
    print(f"   ファイルサイズ: {file_size / 1024 / 1024:.2f} MB")
    
    if file_size > 100 * 1024 * 1024:  # 100MB以上
        print("⚠️ ファイルサイズが大きいため、ダウンロードに時間がかかる場合があります")
        print("💡 Google Driveに保存することも検討してください")
    
    # ダウンロード
    files.download(DB_PATH)
    
    print(f"✅ ダウンロード完了: {DB_PATH}")
    print("\n📌 次のステップ:")
    print("1. このファイルをプロジェクトのルートディレクトリに配置")
    print("2. 管理ページ（admin_app_simple.py）でモデルを学習")
    print("3. 公開ページ（public_app.py）で予測を実行")
    
except FileNotFoundError as e:
    print(f"❌ ファイルエラー: {e}")
    raise
except Exception as e:
    print(f"❌ ダウンロードエラー: {e}")
    print("💡 Google Driveに保存してから手動でダウンロードすることもできます")
    print(f"   コマンド: !cp {DB_PATH} /content/drive/MyDrive/")
    raise

## 🔄 NAR（地方競馬）の処理

NARデータも同様に処理する場合は、MODEを変更して再実行してください。

In [ ]:
# NAR処理の例
MODE_NAR = "NAR"

# 上記のセルをMODE="NAR"で再実行するか、
# または関数化して呼び出す

def process_race_data(mode="JRA", use_venue_features=True):
    """レースデータの処理パイプライン"""
    print(f"\n{'='*60}")
    print(f"モード: {mode}")
    print(f"{'='*60}")
    
    # 1. スクレイピング（省略）
    
    # 2. 特徴量生成
    input_csv = f"database_{mode.lower()}.csv"
    output_csv = f"processed_data_{mode.lower()}.csv"
    
    calculate_features(
        input_csv,
        output_csv,
        lambda_decay=0.2,
        use_venue_features=use_venue_features
    )
    
    # 3. SQL保存
    df = pd.read_csv(output_csv)
    conn = sqlite3.connect(DB_PATH)
    table_name = f"processed_data_{mode.lower()}"
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()
    
    print(f"✅ {mode}データの処理完了")

# JRAとNARの両方を処理
# process_race_data("JRA", use_venue_features=True)
# process_race_data("NAR", use_venue_features=True)

## ✅ 完了

これで以下が完了しました：
1. ✅ スクレイピング
2. ✅ 特徴量生成
3. ✅ SQLiteデータベース作成
4. ✅ データ検証

次のステップ：
1. `keiba_data.db`をダウンロード
2. プロジェクトのルートディレクトリに配置
3. 管理画面でモデルを学習
4. 公開ページで予測を実行